# EDA - Filmes IMDB

O objetivo desse notebook é realizar uma Análise Exploratória de Dados (Exploratory Data Analysis - EDA) sobre o dataset dos 1000 filmes com maior nota do IMDB, retirada do Kaggle no dia 15/02/2021. Esse material não foi feito com o objetivo de apresentar resultados, somente de retirar algumas curiosidades do autor, estudar as ferramentas de análise de dados e demonstrar o processo de pensamento durante o estudo.

Consideração inicial:
Essa não é uma base de dados aleatória, ela é selecionada com um parâmetro claro. Isso é uma análise de "bons filmes na opinião do público", não de filmes em geral.

#### Chamada do dataset e das bibliotecas


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
dados=pd.read_csv('/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv')

#### Observação inicial do dataset

In [ ]:
dados.columns

In [ ]:
dados.head(1)

In [ ]:
dados.info()

In [ ]:
dados.dtypes

Arrumando as colunas que contém elementos numéricos não identificados pelo programa

In [ ]:
dados['Released_Year'].head()

In [ ]:
dados['Released_Year']=pd.to_numeric(dados['Released_Year'], errors='coerce')

In [ ]:
dados['Runtime'].head()

In [ ]:
dados['Runtime']=dados['Runtime'].apply(lambda a: a[0:-3])
dados['Runtime']=pd.to_numeric(dados['Runtime'])

In [ ]:
dados['Gross'].head()

In [ ]:
dados['Gross']=dados['Gross'].apply(lambda a: a.replace(',','') if isinstance(a, str) else a)
dados['Gross']=pd.to_numeric(dados['Gross'], errors='coerce')

In [ ]:
dados.dtypes

#### Analisando as colunas com elementos nulos

In [ ]:
dados[pd.isnull(dados['Meta_score'])].info()

In [ ]:
dados[pd.isnull(dados['Meta_score'])].sample(5)

In [ ]:
dados[pd.isnull(dados['Gross'])].sample(5)

In [ ]:
dados[pd.isnull(dados['Meta_score'])]['No_of_Votes'].describe()

In [ ]:
dados[pd.isnull(dados['Gross'])]['No_of_Votes'].describe()

In [ ]:
dados['No_of_Votes'].describe()

In [ ]:
dados[pd.isnull(dados['Meta_score'])]['Released_Year'].describe()

In [ ]:
dados[pd.isnull(dados['Gross'])]['Released_Year'].describe()

In [ ]:
dados['Released_Year'].describe()

Os buracos na base são filmes relativamente pequenos (1 grau de grandeza menores na quantidade de votos) e em média mais velhos que os dados totais (embora não limitados a filmes antigos). Esses fatores devem ser levados em consideração ao analisar essas variáveis.

#### Hipótese do número de palavras no título e consideração de colunas para estudo

Antes de eliminar colunas, quero testar a hipótese de que filmes com mais palavras no título recebem notas menores ou menos lucro, devido a sequências, que tem foco em quem já assistiu o primeiro filme e tem maior expectativas, e filmes menores, que precisam explicar mais no título devido a menor investimento em divulgação possuirem mais palavras no título.

In [ ]:
dados['Title_words']=dados['Series_Title'].apply(lambda a: len(a.split(' ')))
dados['Title_words'].value_counts()

In [ ]:
dados.corrwith(dados['Title_words'])

Hipótese falha, o número de palavras não tem qualquer relação com a nota e resultado do filme

In [ ]:
dados['Director'].value_counts()

548 diretores diferentes, resultando em uma média menor que 2 filmes por diretor. Não me pareceu uma variável muito interessante

In [ ]:
dados['Genre'].value_counts()

Lembrar de separar os gêneros quando for analisar essa variável, se separar agora vai contaminar pela repetição dos filmes com vários gêneros

## Análise exploratória


In [ ]:
sns.pairplot(dados)

In [ ]:
sns.displot(dados['Meta_score'][pd.notnull(dados['Meta_score'])])

In [ ]:
dados['IMDB_Rating'].describe()

In [ ]:
sns.displot(dados['IMDB_Rating'][pd.notnull(dados['IMDB_Rating'])],kde=False, bins=50)

Percebe-se que 1000 dados não é um número próximo ao total de filmes do IMDB, pois a dispersão da nota do público ainda apresenta tendência de crescimento, ignorando-se o valor mínimo de 7,6, que só estaria completo caso houvesse uma grande coincidência.

In [ ]:
sns.heatmap(dados.corr(),annot=True, cmap="vlag", vmin=-1)

Algumas observações a se fazer:

- A nota nem do público (IMDB_Rating) nem da crítica (Meta_score) influenciam na receita (Gross) dos filmes, o que demonstra que sucesso econômico e de opinião não são correlatos, pelo menos entre os 1000 filmes com melhor nota do público 
- Tanto a nota da crítica quanto a do público tende a aumentar para filmes mais velhos, o que faz sentido ao se observar que o filtro do tempo já foi aplicado, e os filmes antigos com menor qualidade não são tão visualizados e nem estariam na lista dos filmes com maior nota do IMDB.
- No caso da nota da crítica esse fator é aumentado, pois ela é feita não somente com o filme em si, mas mesmo que inconscientemente o sucesso prévio é um fator levado em conta, enquanto as análises de filmes pré-lançamento não possuem esses dados. (Esse valor pode estar contaminado pelas características dos dados faltantes vistos anteriormente)
- Filmes com maior duração que entram nessa lista também tendem a ser levemente melhores em todos os parâmetros com excessão da nota da crítica, com a qual não apresenta correlação.

#### Hipótese sobre número de votos

O número de votos poderia ser expresso pelo número de pessoas com acesso ao filme multiplicado pelas razão dessas que possuem desejo de votar. Como o acesso também é medido pela receita (Gross) e o desejo de votar é influenciado pela nota do público (poucas pessoas sem tal costume entram nesses sites para colocar uma nota mediana, quando os filmes surpreendem positiva ou negativamente há maior desejo de expressar essa opinião). 

Para ver se faz algum sentido essa hipótese, vou analisar a relação entre os filmes com notas menores e o número de votantes, ver se a correlação entre essas variáveis permanece nas regiões mais intermediárias

In [ ]:
sns.jointplot(x='No_of_Votes', y='IMDB_Rating', data=dados)

In [ ]:
dados['IMDB_Rating'].describe()

In [ ]:
sns.heatmap(dados[dados['IMDB_Rating']<=7.9].corr(),annot=True, cmap="vlag", vmin=-1)

Realmente a correlação que se reduz significativamente nos filmes com notas menores. Como isso pode-se dar também pela atração das pessoas que utilizam a plataforma rotineiramente (e que portanto possuem maior probabilidade de dar sua opinião nos filmes que assistiram) a ver os filmes com maior nota, ou também a algum outro fator desconhecido, não posso validar a hipótese, mas ela permanece uma alternativa para o motivo dessas correlações.

### Analise dos Gêneros

In [ ]:
dados_gen=dados
dados_gen['Genre']=dados_gen['Genre'].apply(lambda a: a.split(', '))
dados_gen=dados_gen.explode('Genre')
dados_gen.head()

In [ ]:
dados_gen['Genre'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
fig= sns.countplot(x=dados_gen['Genre'], ax=ax, order = dados_gen['Genre'].value_counts().index)

Mais de 70% dos filmes apresentam "Drama" como gênero, o que demonstra que a utilidade desse gênero para segmentação se perdeu dentro do IMDB. 

Para a análise por gênero, separamos os demais gêneros que apresentam pelo menos 100 filmes, ignorando drama.

In [ ]:
dados_gen[dados_gen['Genre']=='Mystery'].info()

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(16,30))
axs[0, 0].set_title('Comedia')
sns.heatmap(dados_gen[dados_gen['Genre']=='Comedy'].corr(),annot=True, cmap="vlag", vmin=-1, ax=axs[0,0])
axs[0, 1].set_title('Crime')
sns.heatmap(dados_gen[dados_gen['Genre']=='Crime'].corr(),annot=True, cmap="vlag", vmin=-1, ax=axs[0,1])
axs[1, 0].set_title('Aventura')
sns.heatmap(dados_gen[dados_gen['Genre']=='Adventure'].corr(),annot=True, cmap="vlag", vmin=-1, ax=axs[1,0])
axs[1, 1].set_title('Ação')
sns.heatmap(dados_gen[dados_gen['Genre']=='Action'].corr(),annot=True, cmap="vlag", vmin=-1, ax=axs[1,1])
axs[2, 0].set_title('Thriller')
sns.heatmap(dados_gen[dados_gen['Genre']=='Thriller'].corr(),annot=True, cmap="vlag", vmin=-1, ax=axs[2,0])
axs[2, 1].set_title('Romance')
sns.heatmap(dados_gen[dados_gen['Genre']=='Romance'].corr(),annot=True, cmap="vlag", vmin=-1, ax=axs[2,1])

Observações:
- Ao contrário dos outros gêneros com mais de 100 filmes entre os 1000, os filmes de aventura não estão seguindo a tendência de aumentarem de duração com o passar dos anos. Eles também foram o gênero que mais aumentou de receita média com o decorrer do tempo, o que mostra que, embora possuam correlação, essas duas variáveis não demonstram a princípio ter relação causal direta.

- Estranhamente, filmes de romance mais longos foram os únicos entre os analisados a não se mostrar melhores nem para a opinião da crítica e nem dos consumidores, mas apresentaram o maior aumento de receita em relaçãos aos filmes mais curtos. Eles também foram os únicos a não apresentar aumento de receita média com o passar dos anos.

- Embora no global a receita não demonstrou relação com a nota do público, nos filmes de ação, aventura e crime houve uma leve correlação positiva.

## Fim

Obrigado quem leu até aqui. Feedbacks são bem vindos.